# Algorithm Implementation
Author: Edward Schmuhl

---

This notebook shows the algorithm function

In [1]:
import spacy
# Conditionidentification
path_to_ner = "./models/condition_identification_model"
conditionidentification = spacy.load(path_to_ner)

# pretrained Spacy model for Statementextraction
nlp = spacy.load("en_core_web_trf")

# Statementclassification
path_to_textcat = "./models/statement_classification_model"
statement_classifier = spacy.load(path_to_textcat)

In [2]:
def classify_statement(_statement):
    statement_doc = statement_classifier(_statement)
    classifcation = "None"
    value = 0
    
    for cat in statement_doc.cats:
        if statement_doc.cats[cat] > value:
            value = statement_doc.cats[cat]
            classifcation = cat
    
    return classifcation

def extract_classify_Statement(text):
    returnList = []
    condition_Doc = conditionidentification(text)
    doc = nlp(text)
    
    # Conditionidentification
    conditionList = []
    for i in range(0,len(doc)):
        try:
            if(condition_Doc[i].ent_type_ == "CONDITION" and condition_Doc[i].ent_iob_ == "B"):
                index = i
                
                # Conditionidentification
                condition = condition_Doc[i].text
                conditionList.append(condition)
                for k in range(i,len(doc)):
                    if condition_Doc[k].ent_iob_ == "I":
                        condition += " "+condition_Doc[k].text
                        conditionList.append(condition_Doc[k].text)
                        conditionList.append(condition)
                        if doc[k].pos_ == "NOUN" and doc[i].pos_ != "NOUN":
                            index = k

                # Statementextraction        
                statement = ""
                ancestorList = []
                for ancestor in doc[index].ancestors:
                    if ancestor.text not in conditionList:
                        ancestorList.append(ancestor.text)
                for ancestor in reversed(ancestorList):
                    statement += ancestor+" "
                if statement != "":
                    statement += "condition"
                    
                    # Statementclassification
                    statement_class = classify_statement(statement)
                    returnList.append((statement,condition,statement_class))
        except IndexError:
            print(i,doc.text)
            continue
    return returnList

In [3]:
print(extract_classify_Statement("This helped my migraine but caused headache"))

[('helped condition', 'migraine', 'IMPROVED'), ('helped caused condition', 'headache', 'WORSEN')]
